Some of the stages of desiging the network.

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import optimizers
from keras import initializers
from keras.layers import Activation, Dense
from keras.initializers import glorot_normal, he_normal
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

Using TensorFlow backend.


In [ ]:
file = 'c:/users/Anana/Desktop/Ana/sensores/data/SSST/work_main_complete.csv'
# load dataset
dataframe = pd.read_csv(file)
dataframe = dataframe[['label','HRbias','BRbias','HRbiasV','BRbiasV','deltaHRbias','deltaBRbias', 'neu',  'hap']]
dataframe = dataframe.dropna().reset_index(drop=True)
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[1:,1:].astype(float)
Y = dataset[1:,0]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
target_names = ['calm', 'stress']

nTrain=int(0.7*len(X))
X_train = X[:nTrain,:]
y_train = encoded_Y[:nTrain]
X_test = X[nTrain:,:]
y_test = encoded_Y[nTrain:]


X_std_train = np.nan_to_num(X_train)
X_std_test = np.nan_to_num(X_test)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_std_train)
X_train = StandardScaler().fit_transform(rescaledX_train)
rescaledX_test = scaler.fit_transform(X_std_test)
X_test = StandardScaler().fit_transform(rescaledX_test)

#### activation function

In [ ]:
neu = ['tanh', 'relu']

def create_model(neu):
    
    model = Sequential()
    model.add(Dense(50, input_dim=8, kernel_initializer='normal', activation=neu))
    #model.add(Dropout(0.3))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    optimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

for n in neu:
    model = create_model(n)
    history = model.fit(X_train, y_train, batch_size=50, epochs=350,
          verbose=0, validation_data=(X_test, y_test))

    plot_history(history)

    prediction = model.predict(X_test)
    y_pred = np.empty([len(prediction)], dtype=int)
    for i,value in enumerate(prediction):
        if value>0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
            
    cnf_matrix = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=False,
                              title='Confusion matrix')
    
    score_test = model.evaluate(X_test, y_test, verbose=0)
    score_train = model.evaluate(X_train, y_train, verbose=0)
    
    print('Test loss:', score_test[0],'\n Test accuracy:', score_test[1])
    print('Train loss:', score_train[0],'\n Train accuracy:', score_train[1])

#### weight init

In [ ]:
wei = ['normal', 'glorot_normal']

def create_model(wei):
    
    model = Sequential()
    model.add(Dense(50, input_dim=8, kernel_initializer=wei, activation='relu'))
    #model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer=wei, activation='sigmoid'))
    optimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

for w in wei:
    model = create_model(w)
    history = model.fit(X_train, y_train, batch_size=50, epochs=350,
          verbose=0, validation_data=(X_test, y_test))

    plot_history(history)

    prediction = model.predict(X_test)
    y_pred = np.empty([len(prediction)], dtype=int)
    for i,value in enumerate(prediction):
        if value>0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
            
    cnf_matrix = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=False,
                              title='Confusion matrix')
    
    score_test = model.evaluate(X_test, y_test, verbose=0)
    score_train = model.evaluate(X_train, y_train, verbose=0)
    
    print('Test loss:', score_test[0],'\n Test accuracy:', score_test[1])
    print('Train loss:', score_train[0],'\n Train accuracy:', score_train[1])

#### Neurons and layers

- try with/without regularazation and batch norm

In [ ]:
#layer = [1,2,3,4,5,6]
neurons = [200,250,300,350,400]


def create_model(neu):
    
    model = Sequential()
    model.add(Dense(neu, input_dim=8, kernel_initializer=he_normal()))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(neu, kernel_initializer=he_normal()))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(neu, kernel_initializer=he_normal(), activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, kernel_initializer=he_normal(), activation='sigmoid'))
    optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

scores_n = pd.Dataframe()
for l in layers:
    for neu in neurons:
        model = sequential()
        for i in range(0,l):
            model.add(Dense(neu, input_dim=8, kernel_initializer=he_normal()))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(Dropout(0.3))
        optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        print(l,neu)
        plot_history(history)

        prediction = model.predict(X_test)
        y_pred = np.empty([len(prediction)], dtype=int)
        for i,value in enumerate(prediction):
            if value>0.5:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
                
        score_test = model.evaluate(X_test, y_test, verbose=0)
        score_train = model.evaluate(X_train, y_train, verbose=0)
        col = str(l)+'n'+str(neu)
        scores_n[col] = [score_test, score_train]


#### learning rate and reg

In [2]:
#lr = np.random.uniform(low=0.0007, high=0.005, size=(15,))
#reg=np.random.uniform(low = 0.3, high= 0.5, size=(5))

lr = [0.0009, 0.001, 0.002,0.0025]
reg = [0.32,0.33,0.34]

def create_model(lr,reg):
    
    model = Sequential()
    model.add(Dense(200, input_dim=8, kernel_initializer=he_normal()))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(reg))
    model.add(Dense(200, kernel_initializer=he_normal()))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(reg))
    model.add(Dense(200, kernel_initializer=he_normal(), activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(reg))
    model.add(Dense(1, kernel_initializer=he_normal(), activation='sigmoid'))
    optimizer = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

scores=[]
for l in lr:
    for r in reg:
        model = create_model(l, r)
        history = model.fit(X_train, y_train, batch_size=50, epochs=350,
              verbose=0, validation_data=(X_test, y_test))
        
        print(l,r)
        plot_history(history)
    
        prediction = model.predict(X_test)
        y_pred = np.empty([len(prediction)], dtype=int)
        for i,value in enumerate(prediction):
            if value>0.5:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
        
        score_test = model.evaluate(X_test, y_test, verbose=0)
        score_train = model.evaluate(X_train, y_train, verbose=0)
        
        scores.append([l,r,'Test:',score_test,'Train:',score_train])
         